In [1]:
import os

root_directory = "./news_dataset"

unique_docs_per_category = {}
global_seen_docs = {}
untitled_folder_count = 0
unique_files = set()
all_files = []
content_by_category = {}
for root, dirs, files in os.walk(root_directory):
    if os.path.basename(root) == "Untitled Folder":
        untitled_folder_count += 1
        continue
    relative_path = os.path.relpath(root, root_directory)
    path_components = ["Root"] if relative_path == "." else relative_path.split(os.sep)
    for component in path_components:
        if component == "Untitled Folder" or component == "Root":
            continue
        if component not in unique_docs_per_category:
            unique_docs_per_category[component] = 0
            global_seen_docs[component] = set()
            content_by_category[component] = {'titles': [], 'bodies': []}
    files.sort(key=lambda x: int(os.path.splitext(x)[0]))
    for file in files:
        unique_files.add(file)
        all_files.append(file)
        file_path = os.path.join(root, file)
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            title = lines[0].strip()
            body = ''.join(lines[1:]).strip()
        for category in path_components:
            if category != "Untitled Folder" and file not in global_seen_docs[category]:
                unique_docs_per_category[category] += 1
                global_seen_docs[category].add(file)
                content_by_category[category]['titles'].append(title)
                content_by_category[category]['bodies'].append(body)

In [2]:
print("Number of categories: " + str(len(unique_docs_per_category.keys())))

Number of categories: 113


In [3]:
print("Number of documents: " + str(len(all_files)))

Number of documents: 17599


In [4]:
print("Number of unique documents: " + str(len(unique_files)))

Number of unique documents: 6539


In [5]:
import pandas as pd

# Assuming `content_by_category` is the dictionary you're working with
data_samples = []
for category in content_by_category.keys():
    contents = content_by_category[category]
    title, body = contents['titles'][0],contents['bodies'][0]
    displayed_body = (body[:47] + '...') if len(body) > 50 else body
    data_samples.append({"Title": title, "Class": category, "Body": displayed_body})

df_samples = pd.DataFrame(data_samples)

display(df_samples)


,Title,Class,Body
0,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,اجتماعی,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
1,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,اطلاعاتی,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
2,مشاور اوباما: تجمع حوثی‌ها در صنعا را محکوم می...,جو فرهنگی حاکم بر کشور,خبرگزاری تسنیم: مشاور ارشد رئیس جمهور آمریکا د...
3,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را...,فرهنگی امنیتی,تهران- ایرنا- اطلاعات حاکی از آن است که آمریکا...
4,آمریکا پنج میلیارد رکورد اطلاعاتی ایرانی ها را...,تهدیدات اجتماعی و فرهنگی,تهران- ایرنا- اطلاعات حاکی از آن است که آمریکا...
...,...,...,...
108,گروگانگیری در شعبه بانک انصار شوش با دستگیری گ...,مبارزه با گروگانگیری,دزفول- ایرنا- فرد مسلحی که صبح پنجشنبه با ورود...
109,هشدار رئیس‌جمهور چین نسبت به تهدید‌های فزاینده...,مسولان عالی تصمیم گیر کشوری و لشگری,رییس‌جمهوری چین با تاکید بر اینکه چین با تهدید...
110,آمریکا برای حمله به سوریه آماده می‌شود,پدافند هوایی,وزیر دفاع آمریکا اعلام کرد، پنتاگون در حال است...
111,پنتاگون در آستانه تعطیل دولت، ۵ میلیارد دلار ت...,عملیات روانی و تهاجم فرهنگی دشمن,ب پیش از فرستاده شدن ۸۰۰ هزار کارمند دولت آمری...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import numpy as np
import hazm
import pandas as pd



normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
punctuations = [')', '(', '>', '<', "؛", "،", '{', '}', "؟", ':', "–", '»', '"', '«', '[', ']', '"', '+', '=', '?', '/',
                '//', '\\', '|', '!', '%', '&', '*', '$', '#', '؟', '*', '.', '_', '']
preprocessed_docs = []
categories = []
stopwords_list = hazm.stopwords_list()
for category in content_by_category.keys():
    category_text = ""
    for doc in content_by_category[category]['bodies']:
        normalized_text = normalizer.normalize(doc)
        tokens = hazm.word_tokenize(normalized_text)
        lemmatized_text = ""
        for token in tokens:
            if token in stopwords_list or token in punctuations:
                continue
            lemmatized_token = lemmatizer.lemmatize(token)
            lemma = lemmatized_token.split('#')[1] if '#' in lemmatized_token else lemmatized_token
            lemmatized_text += lemmatized_token + " "
        category_text += lemmatized_text + " "

    preprocessed_docs.append(category_text.strip())
    categories.append(category)




In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import numpy as np
import hazm
import pandas as pd

tfidf_vectorizer = TfidfVectorizer(max_features=10000,  ngram_range=(1,2),sublinear_tf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_docs)
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

print(df_tfidf)


NameError: name 'preprocessed_docs' is not defined

In [51]:
import pandas as pd

# Adjust display settings
pd.set_option('display.max_colwidth', None)  # For pandas versions >= 1.0
# For older versions of pandas, use 'display.max_colwidth', -1

data_samples = []
for index, category in enumerate(categories):
    value = df_tfidf.iloc[index, :]
    contents = content_by_category[category]
    top_scores = value.sort_values(ascending=False).head(10)
    top_words = top_scores.index
    
    # Convert the index object 'top_words' to a string separated by commas
    top_words_str = ', '.join(top_words)
    
    title = contents['titles'][0]
    
    # Append the 'top_words_str' instead of 'top_words' index object
    data_samples.append({"Title": title, "Class": category, "Key words": top_words_str})

df_samples = pd.DataFrame(data_samples)

# Display the DataFrame
display(df_samples)


آباد


In [75]:
import pandas as pd

# Assuming 'tfidf_vectorizer' is your TfidfVectorizer instance and it's already been fit to your dataset
feature_names = tfidf_vectorizer.get_feature_names_out()
idf_scores = tfidf_vectorizer.idf_

# Pairing feature names with their IDF scores and sorting by IDF score in ascending order
sorted_features_by_idf = sorted(zip(feature_names, idf_scores), key=lambda x: x[1])

# Selecting the top N words with the lowest IDF scores
top_n = 100
lowest_idf_words = sorted_features_by_idf[:top_n]

# Creating a DataFrame from the top N lowest IDF scores
df_lowest_idf = pd.DataFrame(lowest_idf_words, columns=['Word', 'IDF Score'])

# Display the DataFrame
display(df_lowest_idf)


Top 10 TF-IDF Scores for the First Document:
ایران: 0.47314137847428506
کن: 0.3782210402001229
شو: 0.303014926029056
کشور: 0.24387225371977037
گو: 0.1467614461008199
گزارش: 0.1423805074112432
ده: 0.11828534461857126
باش: 0.11755518817030848
مخدر: 0.11253964667190496
مواد: 0.10626042693172154
سال: 0.09857112051547605
قاچاق: 0.09330738473932354
موادمخدر: 0.09270612990895831
مرز: 0.08542830444674591
آمریکا: 0.08396799155022033
نیرو: 0.08031720930890641
نظام: 0.07666642706759248
ترکیه: 0.07374580127454135
پاکستان: 0.0709501071871688
سوریه: 0.07009501903322742
امنیت: 0.07009501903322742
منطقه: 0.06790454968843905
گیر: 0.06425376744712513
دولت: 0.06425376744712513
توان: 0.06425376744712513
همکار: 0.06425376744712513
سپاه: 0.06279345455059956
سلاح: 0.06133314165407399
اسلام: 0.0606029852058112
افغانستان: 0.059872828757548416
